# <font color = 'orange'> Model Evaluation

---

In [1]:
import os

In [2]:
%pwd

'e:\\PW Skills - Data Science\\Projects\\09_Chest_Cancer_Classification\\notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\PW Skills - Data Science\\Projects\\09_Chest_Cancer_Classification'

---

In [5]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml

secrets = read_yaml(SECRETS_FILE_PATH)
mlflow = secrets.mlflow

os.environ["MLFLOW_TRACKING_URI"]=f"{mlflow.MLFLOW_TRACKING_URI}"
os.environ["MLFLOW_TRACKING_USERNAME"]=f"{mlflow.MLFLOW_TRACKING_USERNAME}"
os.environ["MLFLOW_TRACKING_PASSWORD"]=f"{mlflow.MLFLOW_TRACKING_PASSWORD}"

[2024-04-23 15:51:20,858: INFO: common: yaml file: secrets\secrets.yaml loaded successfully]


---

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    testing_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import create_directories, read_yaml
from pathlib import Path


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        secrets_file_path = SECRETS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secrets = read_yaml(secrets_file_path)
        create_directories([self.config.artifacts_root])
        
    
    def get_evaluation_config(self) -> EvaluationConfig:
        mlflow_secrets = self.secrets.mlflow
        params = self.params
        
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            testing_data=Path(self.config.evaluation.testing_data),
            mlflow_uri=mlflow_secrets.MLFLOW_TRACKING_URI,
            all_params=params,
            params_image_size=params.IMAGE_SIZE,
            params_batch_size=params.BATCH_SIZE
        )
        
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from keras.preprocessing.image import ImageDataGenerator
from chest_cancer_classifier.utils.common import save_json


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    
    def __test_generator(self):
        test_datagenerator = ImageDataGenerator(
            rescale = 1./255,
            validation_split=0.30
        )

        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.testing_data,
            subset="validation",
            shuffle=False,
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.__test_generator()
        self.score = self.model.evaluate(self.test_generator)
        self.save_score()


    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {
                    "loss": self.score[0],
                    "accuracy": self.score[1]
                }
            )
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [9]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2024-04-23 15:51:33,417: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-23 15:51:33,420: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-23 15:51:33,427: INFO: common: yaml file: secrets\secrets.yaml loaded successfully]
[2024-04-23 15:51:33,429: INFO: common: created directory at: artifacts]


Found 102 images belonging to 2 classes.
8/8 [==============================] - 20s 2s/step - loss: 1.2285 - accuracy: 0.5000
[2024-04-23 15:51:55,589: INFO: common: json file saved at: scores.json]


2024/04/23 15:51:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-04-23 15:52:00,562: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
[2024-04-23 15:52:01,971: INFO: builder_impl: Assets written to: C:\Users\darsh\AppData\Local\Temp\tmp0qidbnpn\model\data\model\assets]


e:\PW Skills - Data Science\Projects\09_Chest_Cancer_Classification\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


---